<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd # библиотека Pandas
import numpy as np # библиотека NumPy
from sklearn.linear_model import LinearRegression # линейная регрессия
from sklearn.metrics import r2_score # r2 метрика

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Датасет состоит из 5 тысяч строк и 5 столбцов. Обучающие признаки: пол, возраст, зарплата, количество членов семьи. Целевой признак — количество страховых выплат.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Требуется зашифровать данные так, чтобы модель линейной регрессии не потеряла своих качеств. Предлагается умножить признаки на матрицу, которая обратима. Вопрос: изменится ли качество модели на измененных признаках?

**Ответ:** не изменится.

**Обоснование:** запишем новую формулу.

$$
w = ((XP)^T (XP))^{-1} (XP)^T y
$$

Известно, что транспонирование произведения матриц равно произведению транспонированных матриц, взятых в обратном порядке, т.е.:

$$
(AB)^T = B^T A^T
$$

Перепишем уравнение:

$$
w = (P^T X^T (XP))^{-1} P^T X^T y
$$

Также известно свойство обратных матриц:

$$
(AB)^{-1} = B^{-1} A^{-1}
$$

И свойство ассоциативности матриц:

$$
A (BC) = (AB) C
$$

Перепишем уравнение:

$$
w = P^{-1}(X^T X)^{-1} P^{T^{-1}} P^T X^T y
$$

Матрица, умноженная на обратную, обладает следующим свойством:

$$
A A^{-1} = A^{-1} A = E
$$

В итоге уравнение принимает вид:

$$
w = P^{-1} (X^T X)^{-1} X^T y
$$


В итоге получается, что в формуле предсказания появляется две новых переменных, которые дают в произведении E:

$$
a = XP P^{-1} w = X E w
$$

А как известно:

$$
X E = X
$$

Следовательно:

$$
a = X w
$$

Что и требовалось доказать.

Разница формулы весов для обычных и преобразованных признаков.

$$
w = (X^T X)^{-1} X^T y
$$$$
w_P = P^{-1} (X^T X)^{-1} X^T y
$$

Новая формула содержит коэффициент.

$$
P^{-1}
$$

Данный коэффициент сокращается во время нахождения целевого признака,

## Алгоритм преобразования

**Алгоритм**

Возьмем случайную матрицу размеров 4 х 4 (только квадратная матрица имеет обратную), и умножим признаки на нее, далее обучим модель на новых признаках и сравним качество модели с оригинальной.

**Обоснование**

Алгоритм обосновывается формулой, которая была выведена выше. Ниже можно рассмотреть пример, которые иллюстрирует формулу.

Выделим обучающие признаки и умножим на случайную матрицу 4 х 4, а далее на обратную ей матрицу, мы должны получить исходные данные.

In [4]:
np.random.seed(75)
test_features = data.drop(columns='Страховые выплаты').head(4)
test_matrix = np.random.rand(4,4)
inv_test_matrix = np.linalg.inv(test_matrix)

In [5]:
display(test_features)
display((np.round(test_features @ test_matrix @ inv_test_matrix)).astype('int64'))

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2


,0,1,2,3
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2


Как можно заметить, данные после преобразование не изменились, формула работает.

## Проверка алгоритма

In [6]:
features = data.drop(columns='Страховые выплаты', axis=1)
target = data['Страховые выплаты']
model_original = LinearRegression()
model_original.fit(features, target)
print('R2 оригинальной модели:', r2_score(target, model_original.predict(features)))

R2 оригинальной модели: 0.4249455028666801


Создадим матрицу 4 х 4 и проверим ее обратимость.

In [7]:
matrix = np.random.rand(4,4)
np.linalg.inv(matrix)

array([[ -2.54773559,  -1.68021092,   0.19503476,   3.27472764],
       [-16.29074687, -20.43153602,   3.55084425,  22.5865093 ],
       [ 25.11292394,  32.1571056 ,  -3.84066438, -36.80061206],
       [ -2.39036068,  -5.36595151,   0.51899692,   5.80976917]])

Матрица обратима, умножим ее на признаки и проверим модель.

In [8]:
new_features = features @ matrix
model_matrix = LinearRegression()
model_matrix.fit(new_features, target)
print('R2 обновленной модели:', r2_score(target, model_matrix.predict(new_features)))

R2 обновленной модели: 0.4249455028668023


Метрика практически не изменилась, при этом новые признаки выглядят так:

In [9]:
new_features

,0,1,2,3
0,37096.194474,48637.935027,38598.000194,34491.891247
1,28435.094125,37264.979140,29574.176738,26428.901763
2,15717.181938,20594.273639,16344.297849,14606.074515
3,31175.183025,40889.261084,32447.450605,28994.634380
4,19526.402573,25594.669298,20312.036095,18151.749432
...,...,...,...,...
4995,26699.828982,35007.371359,27781.025105,24825.726585
4996,39180.738695,51382.201697,40774.657400,36435.617044
4997,25347.133570,33241.358423,26378.868168,23572.241872
4998,24453.835567,32065.224929,25446.109567,22740.017813


## Вывод

В данной работе требовалось зашифровать данные клиентов страховой фирмы так, чтобы не раскрывать информацию о клиенте. При этом в данных существует целевой признак, который должна рассчитывать модель линейной регрессии, важно, чтобы после шифрования модель могла продолжать работать.

Задача решается умножением обучающих признаков на обратимую матрицу. Характер предсказаний на новых данных не меняется, что было строго математически доказано и проверено.